In [1]:
#Import libraries
from bqplot import *
from ipywidgets import *
import ipywidgets as widgets
from IPython.display import clear_output
import pandas as pd

# Frequency planning tool for RFSoC devices

This Jupyter notebook contains the following:
    
    1) Digital Up Conversion Spectrum (DUC)
    2) RF Digital to Analogue frequency plan chart (RF-DAC)
    3) RF Analogue to Digital frequency plan chart (RF-ADC) 
    4) Digital Down Conversion Spectrum (DDC)
     

These tools will allow the user to create an in depth frequency plan for any RF project

# User guide

In all charts sliders are used to select the Sampling frequency, Input frequency and the Bandwidth of the input signal. 
In the DUC and DDC charts sliders are also available to set the Frequency of the NCO and the level of noise (in dB) in the Harmonic spurs HD2 and HD3. 
The DDC chart features radio buttons that allow the user to select the number of interleaved ADC's available and the Decimation factor.
The display numbers button will show the dataframe created for the graph with all the relevant values for the respective chart. 
Hover the mouse over the markers on the charts to display a hovertip with the relevant information. 

# Colour code

Each of the frequency charts in this application adhere to the following colour scheme. Note that this is not a frequency chart, it is here to display the colour scheme only.

In [2]:
## ATTENTION! 
## The following code segment is for plotting a colour key only and has no bearing over any of Frequency planners functionality

input_frequency = [100,200]
Nyquist = 1500
HD2 = [300,1000]
I_HD2 = [300,1000]
I_HD3 = [300,1000]
HD3 = [300,1000]
HD4 = [300,1000]
HD5 = [300,1000]


X_spur = [HD2,HD3,HD4,HD5,I_HD2,I_HD3]  
Y_spur = [[2,2],[3,3],[4,4],[5,5],[2.5,2.5],[3.5,3.5]]

x_sc = LinearScale()
y_sc = LinearScale()

ax_x = Axis(label='', scale=x_sc, tick_format='0.0f')
ax_y = Axis(label='', scale=y_sc,
               orientation='vertical', tick_format='0.2f')   

#Create lines for plotting
Frequency_plot = Lines(x=input_frequency,y=[5,5],scales={'x': x_sc, 'y': y_sc}, colors=['blue'],
                   labels=['Frequency in'],display_legend=True)
Frequency_plot2 = Lines(x=[100,100],y=[0,5],scales={'x': x_sc, 'y': y_sc}, colors=['blue'])
Frequency_plot3 = Lines(x=[200,200],y=[5,0],scales={'x': x_sc, 'y': y_sc}, colors=['blue'])
Ny_plot = Lines(x=[Nyquist,Nyquist],y=[0,5],scales={'x': x_sc, 'y': y_sc}, colors=['red'],labels=['Nyquist'],
                display_legend=True)
spur_plot = Lines(x=X_spur,y=Y_spur,scales={'x': x_sc, 'y': y_sc}, colors=['cyan','pink','purple','yellow','orange','green'],
                  labels=['HD2','HD3','HD4','HD5','HD2 interleaved','HD3 interleaved'],display_legend=True)

   
fig = Figure(title='Colour code',axes=[ax_x, ax_y], marks=[Frequency_plot,Frequency_plot2,Frequency_plot3,Ny_plot,spur_plot],
                     display_legend=True)


    
display(fig)

Figure(axes=[Axis(scale=LinearScale(), tick_format='0.0f'), Axis(orientation='vertical', scale=LinearScale(), …

# 1) DUC Frequency chart

The following spectrum will display the relevevant information for the Digital up conversion process. If mix mode is on Nyquist zone 2 will show the relevant spurs. If normal mode is on Nyquist zone 1 will show the relevant spurs. The spectrum shows both Nyquist zones so user does not need to make changes depending on which mode they operate.

In [3]:
#Function for calculating spurs that are shown on DUC graph

def DUC_spur_calculator(Fs, Frequency_in, NCO):
    
    Nyquist = float(Fs / 2)          #Calculate Nyquist rate
    Fundamental = NCO + Frequency_in #Calculate Fundamental frequency
    F_image_calc = Fundamental - Fs  #Calculate image of fundamental frequency

    #Mirror image if negative
    if Fundamental < 0:
        Fundamental = Fundamental * -1
    if F_image_calc < 0:
        F_image_calc= F_image_calc*-1

    HDlist = [] #Create a list for storing loop values
    
    # Calculate K variable, K increases as conditions are met
    
    for n in range (2,4,1):    
        
        if Fundamental*n < Nyquist:
            K = 0
        
        elif Fundamental*n <= Fs + Nyquist:
            K = 1
    
        elif Fundamental*n <= 2*Fs + Nyquist:
            K = 2
        
        elif Fundamental*n <= 3*Fs + Nyquist:
            K = 3
        
        elif Fundamental*n <= 4*Fs + Nyquist:
            K = 4
            
        elif Fundamental*n <= 5*Fs + Nyquist:
            K = 5
        else:
            K = 6
        
        HDn = (Fundamental *n) - (Fs*K)
        
        if HDn < 0:
            HDn = HDn*-1


        if HDn < 0:        #IF HDn is negative, mirror result 

                    Mirror_HDn = HDn * -1;
                 
                    temp_list = (Mirror_HDn) 
                    HDlist.append(temp_list)        #Update list with calculated values                 
        
        elif HDn > Fs:
            stopprocess = False

            while stopprocess == False:
                if HDn > Fs:
                    HDn = HDn-Fs
            
            #Set to true when value is below decimated Nyquist
                if HDn <= Fs:
                    stoprocess = True
                    temp_list = (HDn)
                    HDlist.append(temp_list) 
                    
                    break
        
        else:
                    temp_list = (HDn)
                    HDlist.append(temp_list)        #Update list with calculated values

    
    # Values that are returned from function
    HD2_value = HDlist[0]
    HD3_value = HDlist[1]
    HD2_image = Fs - HD2_value
    HD3_image = Fs - HD3_value
    
    return [HD2_value, HD3_value,HD2_image,HD3_image]


In [4]:
#Function to plot DUC 
def Plot_DUC(Fs,Frequency_in,NCO,HD2_noise,HD3_noise):
   
    Nyquist = float(Fs / 2)      #Calculate Nyquist rate
    Fundamental = NCO + Frequency_in #Calculate fundamental frequnecy
    F_image = Fs - Fundamental #Calculate image of fundamental frequency

    #if  Nyquist <= F_image:    
    #    F_image = F_image-Fs
     #   F_image = F_image *-1
    
    if F_image < 0:
        F_image = F_image*-1
    
    if Fundamental < 0:
        Fundamental = Fundamental*-1
    #Call spur calculator
    HD = DUC_spur_calculator(Fs,Frequency_in,NCO)
    
    #Get values from spur calculator
    HD2 = HD[0]
    HD3 = HD[1]
    HD2_image = HD[2]
    HD3_image = HD[3]
    
    #Create plotting data
    x_plot_HD2 = [HD2, HD2]
    x_plot_HD3 = [HD3, HD3]
    x_plot_HD2_image = [HD2_image, HD2_image]
    x_plot_HD3_image = [HD3_image, HD3_image]
    y_plot_HD2 = [-1*HD2_noise,-100]
    y_plot_HD3 = [-1*HD3_noise,-100]
    fund_x = [Fundamental,Fundamental]
    fim_x = [F_image, F_image]
    Fs_x = [Fs,Fs]
    fund_y = [0,-100]
    Nyquist1 = [Nyquist,Nyquist]
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Amplitude(dB)', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting
    HD2_plot = Lines(x=x_plot_HD2,y=y_plot_HD2,scales={'x': x_sc, 'y': y_sc}, colors=['cyan'],labels=['HD2'])
    HD3_plot = Lines(x=x_plot_HD3,y=y_plot_HD3,scales={'x': x_sc, 'y': y_sc}, colors=['pink'],labels=['HD3'])
    HD2_image_plot = Lines(x=x_plot_HD2_image,y=y_plot_HD2,scales={'x': x_sc, 'y': y_sc}, colors=['cyan'],labels=['HD2'])
    HD3_image_plot = Lines(x=x_plot_HD3_image,y=y_plot_HD3,scales={'x': x_sc, 'y': y_sc}, colors=['pink'],labels=['HD3'])
    Fundamental_plot = Lines(x=fund_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['blue'],labels=['Fundamental'])
    Fim_plot = Lines(x=fim_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['blue'],labels=['F image'])
    Nyquist_plot = Lines(x=Nyquist1,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'],labels=['Nyquist'])
    Fs_plot = Lines(x=Fs_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'],labels=['Fs'])
   
    #Create tooltips for data 
    Fundamental_tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Fundamental Frequency (MHz)', 'dB level (dB)'])
    Fim_tt = Tooltip(fields=['x'], formats=[''], labels=['Frequency Image (MHz)'])
    tt_HD2 = Tooltip(fields=['x'], formats=[''], labels=['HD2 (MHz)','dB level (dB)'])
    tt_HD3 = Tooltip(fields=['x'], formats=[''], labels=['HD3 (MHz)','dB level (dB)'])
    tt_HD2_image = Tooltip(fields=['x'], formats=[''], labels=['HD2 image (MHz)','dB level (dB)'])
    tt_HD3_image = Tooltip(fields=['x'], formats=[''], labels=['HD3 image (MHz)','dB level (dB)'])
    Nyquist_tt = Tooltip(fields=['x'], formats=[''], labels=['Nyquist Frequency (MHz)'])
    Fs_tt = Tooltip(fields=['x'], formats=[''], labels=['Sampling Frequency (MHz)'])
    
    #Match data to relative tooltips
    HD2_plot.tooltip=tt_HD2
    HD3_plot.tooltip=tt_HD3
    HD2_image_plot.tooltip=tt_HD2_image
    HD3_image_plot.tooltip=tt_HD3_image
    Fundamental_plot.tooltip=Fundamental_tt
    Nyquist_plot.tooltip=Nyquist_tt
    Fs_plot.tooltip=Fs_tt
    Fim_plot.tooltip=Fim_tt
    
    #Create markers for data points
    HD2_plot.marker='square'
    HD3_plot.marker='square'
    HD2_image_plot.marker='square'
    HD3_image_plot.marker='square'
    Fundamental_plot.marker='square'
    Fim_plot.marker='square'
    Fs_plot.marker='square'
    Nyquist_plot.marker = 'square'

    
    fig = Figure(title='DUC Spectrum',axes=[ax_x, ax_y], marks=[HD2_plot,HD3_plot,HD2_image_plot,HD3_image_plot,Fundamental_plot,Fs_plot,Nyquist_plot,Fim_plot],
                     display_legend=True)

    tools=Toolbar(figure=fig)
    
    display(fig, tools)
    
    #Create a data frame for numerical values
    frame2={'':['Fundamental','Image','NCO','HD2','HD2 image','HD3','HD3 image'],
            'Frequency (MHz)':[Fundamental,F_image,NCO,HD[0],HD[2],HD[1],HD[3]]}
 
       
    tempFrame = pd.DataFrame(data=frame2)
    tempFrame.set_index('',inplace = True)    
    #tempFrame.to_csv('ADC.csv', sep=',',index=False)

    #Create button widget
    def clicked(b):
    
    
        with out:
            clear_output(wait=True)
            display(tempFrame)
            
    out = widgets.Output()
    display(out)

    run_button = widgets.Button(
        description = 'Display numbers'
        )
    run_button.on_click(clicked)
    display(run_button)    

  

In [5]:
#Create sliders for input info
Fs_slider = widgets.FloatSlider(value=4000,min=1000,max=4000,step=100,description='Fs:')
Signal_Center_slider = widgets.FloatSlider(value=3500,min=100,max=4000,step=50,description='Signal centre:')
NCO_slider = widgets.FloatSlider(value=0,min=-2000,max=2000, step = 50,description='NCO:')
HD2_slider = widgets.FloatSlider(value=10, min=0, max=100, step = 10,description='HD2 noise:')
HD3_slider = widgets.FloatSlider(value=10, min=0, max=100, step = 10,description='HD3 noise:')

#Create text boxes for input info
Fs_text = widgets.BoundedFloatText(value=4000,min=1000,max=4000,step=0.1,description='Fs:',continuous_update = False)
Signal_Center_text = widgets.BoundedFloatText(value=3500,min=100,max=4000,step=0.1,description='Signal centre:',continuous_update = False)
NCO_text = widgets.BoundedFloatText(value=0,min=-2000,max=2000,step=0.1,description='NCO:',continuous_update = False)
HD2_text = widgets.BoundedFloatText(value=10,min=0,max=100,step=0.1,description='HD2 noise:',continuous_update = False)
HD3_text = widgets.BoundedFloatText(value=10,min=0,max=100,step=0.1,description='HD3 noise:',continuous_update = False)

#Link text boxes and sliders
l = widgets.link((Fs_slider, 'value'), (Fs_text, 'value'))
l = widgets.link((Signal_Center_slider, 'value'), (Signal_Center_text , 'value'))
l = widgets.link((NCO_slider, 'value'), (NCO_text, 'value'))
l = widgets.link((HD2_slider, 'value'), (HD2_text, 'value'))
l = widgets.link((HD2_slider, 'value'), (HD2_text, 'value'))

#Display widget layout
left_box = VBox([Fs_slider, Signal_Center_slider, NCO_slider,HD2_slider,HD3_slider])
right_box = VBox([Fs_text, Signal_Center_text, NCO_text,HD2_text,HD3_text])
sel=HBox([left_box, right_box])
display(sel)

#Interactive plot to link plot function with widgets
out = widgets.interactive_output(Plot_DUC, {'Fs': Fs_slider, 'Frequency_in': Signal_Center_slider, 'NCO': NCO_slider,
                                              'HD2_noise':HD2_slider,'HD3_noise':HD3_slider })
display(out)

Output()

# 2) RF-DAC Frequency chart

The following chart provides the relevant information required for the RFDAC process.

In [6]:
#This function calculates the input signal and its image, to be displayed in the first two Nyquist zones
#Requires 3 input arguments, set from sliders
#Fs = Sampling Frequency, Signal_center = location of center of signal, Signal_bandwidth = Bandwidth of signal

def DAC_Input_signal_calculator(Fs, Signal_center, Signal_Bandwidth):
    
    #Calculate low and high boundaries of signal
    Fin_low = Signal_center - (Signal_Bandwidth/2)
    Fin_High = Signal_center + (Signal_Bandwidth/2)
    
    #Calculate Nyquist rate
    Nyquist = float(Fs/2)
    
    #Calculate K variable
    if Fin_High <= Nyquist:
        K = 0
        
    elif Fin_High <= Fs + Nyquist:
        K = 1
    
    elif Fin_High <= 2*Fs + Nyquist:
        K = 2
        
    elif Fin_High <= 3*Fs + Nyquist:
        K = 3
        
    elif Fin_High <= 4*Fs + Nyquist:
        K = 4
    
    elif Fin_High <= 5*Fs + Nyquist:
        K=5
    
    else:
        K =6
        
    Fin_low_alias = (Fin_low - Fs*K)
    Fin_High_alias = (Fin_High - Fs*K)   
    
    
    if Fin_High_alias < Nyquist and Fin_low_alias > 0:
        Fin_low_1 = Fin_low_alias
        Fin_high_1 = Fin_High_alias
        temp = Nyquist - Fin_high_1
        Fin_low_2 = (temp)+Nyquist
        temp2 = Nyquist - Fin_low_1
        Fin_high_2 = (temp2)+Nyquist
        
          
    elif Fin_low_alias > Nyquist:
        Fin_low_2 = Fin_low_alias
        Fin_high_2 = Fin_High_alias
        temp =  Fin_High_2 - Nyquist
        Fin_low_1 = Nyquist - temp
        temp2 = Fin_low_2 - Nyquist
        Fin_high_1 = Nyquist - temp2
        
        
    elif Fin_low_alias > Fs:
        temp=Fin_low_alias - Fs
        Fin_high_2 = Fs - temp
        temp2 = Fin_High_alias - Fs
        Fin_low_2 = Fs-temp2
        temp3 = Fin_high_2 - Nyquist
        Fin_low_1 = Nyquist - temp3
        temp4 = Fin_low_2 - Nyquist
        Fin_high_1 = Nyquist - temp4
        

    elif Fin_High_alias < 0:
            Fin_low_1 = Fin_High_alias*-1
            Fin_high_1 = Fin_low_alias*-1
            temp = Nyquist - Fin_high_1
            Fin_low_2 = Nyquist + temp
            temp2 = Nyquist - Fin_low_1
            Fin_high_2 = Nyquist + temp2
            
            
    elif Fin_low_alias < 0 and Fin_High_alias > 0:
            Fin_low_1 = 0
            Fin_high_1 = Fin_High_alias
            temp = 0 + Fin_high_1
            Fin_low_2 = Fs - temp
            Fin_high_2 = Fs
                 
   
    elif Fin_low_alias < 0 and Fin_High_alias == 0:
            Fin_low_1 = 0
            Fin_high_1 = Fin_low_alias*-1
            temp = 0 + Fin_high_1
            Fin_low_2 = Fs - temp
            Fin_high_2 = Fs
            
    #Return signal and its alias
    
    if Fin_low_1 < 0:
        Fin_low_1 = 0
    if Fin_high_1 > Nyquist:
        Fin_high_1 = Nyquist
    if Fin_low_2 < Nyquist:
        Fin_low_2 = Nyquist
    if Fin_high_2 > Fs:
        Fin_high_2 = Fs
        
    return [Fin_low_1,Fin_high_1,Fin_low_2,Fin_high_2]
 

In [7]:
#This function will calculate the spurs due to the signal and their images
def DAC_Spur_Calculator(Fs, Signal_center, Signal_Bandwidth):

    #Call DAC inut function
    Input_info = DAC_Input_signal_calculator(Fs,Signal_center,Signal_Bandwidth)
    Nyquist = Fs/2
    #Set low and high boundaries to values from DAC function call
    fin_low = Input_info[0]
    fin_high = Input_info[1]
    fin_high_alias = Input_info[2]
    fin_low_alias = Input_info[3]
    
    #Create list for storing spur values
    HDlist=[]
        
    #For loop for calculating spurs HD2 upto HD5
    for n in range (2,6,1):
       
        if fin_high*n <= Nyquist:
            K = 0
        
        elif fin_high*n <= Fs + Nyquist:
            K = 1
    
        elif fin_high*n <= 2*Fs + Nyquist:
            K = 2
        
        elif fin_high*n <= 3*Fs + Nyquist:
            K = 3
        
        elif fin_high*n <= 4*Fs + Nyquist:
            K = 4
            
        elif fin_high*n <= 5*Fs + Nyquist:
            K = 5
        else:
            K = 6
        
        #Calculate spur locations
        HDn_high = (fin_high *n) - (Fs*K)
        HDn_low = (fin_low *n) - (Fs*K)
    
        #Conditional statements to determine spur location
    
        if HDn_high <= 0 and HDn_low < 0:        #IF HDn is negative, mirror result 

                    Mirror_HDn_low = HDn_high * -1;
                    Mirror_HDn_high = HDn_low * -1;
                    
                    if Mirror_HDn_high > Nyquist:
                        Mirror_HDn_high = Nyquist
                        temp_list = (Mirror_HDn_low,Mirror_HDn_high) #Temporary list to store values
                        HDlist.append(temp_list)
                    
                    elif Mirror_HDn_low < 0:
                        Mirror_HDn_low = 0
                        temp_list = (Mirror_HDn_low,Mirror_HDn_high) #Temporary list to store values
                        HDlist.append(temp_list)
                    else:
                        
                        temp_list = (Mirror_HDn_low,Mirror_HDn_high) #Temporary list to store values
                        HDlist.append(temp_list)        #Update list with calculated values                 
        
        elif  Nyquist < HDn_high and HDn_low < Nyquist:

                    HDn_high = Nyquist
                    temp_list = (HDn_low,HDn_high)
                    HDlist.append(temp_list)
       
        elif HDn_low > 0 and HDn_high < 0:
            HDn_high = HDn_high * -1
            temp_list = (HDn_low,HDn_high)
            HDlist.append(temp_list)
            
        elif HDn_low < 0 and HDn_high > 0:
            HDn_low = 0
            temp_list = (HDn_low,HDn_high)
            HDlist.append(temp_list)
                
        else:
                    temp_list = (HDn_high,HDn_low)
                    HDlist.append(temp_list)        #Update list with calculated values
          
     #Create array to check if list values are greater than Nyquist
    #This is used to set spurs greater than Nyquist = Nyquist, for plotting purposes
    
    HD_High= [HDlist[0][1],HDlist[1][1],HDlist[2][1],HDlist[3][1]]
  
    HD_High_list=[]
    
    for i in HD_High:
        if i <= 0 :
            i = 0 
            temp_list = (i)
            #Update list with calculated values
            HD_High_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
            #Update list with calculated values
            HD_High_list.append(temp_list)
    
    #Create array to check if list values are less than 0
    #This is used to set spurs less than 0 = 0, for plotting purposes
    
    HD_low = [HDlist[0][0],HDlist[1][0],HDlist[2][0],HDlist[3][0]]
    
    
    HD_low_list=[]
    
    for i in HD_low:
        if i > Nyquist:
            i = Nyquist
            temp_list = (i)
            #Update list with calculated values
            HD_low_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
            #Update list with calculated values
            HD_low_list.append(temp_list)

    #Create spur values with relative list values
    HD2_value = [HD_High_list[0], HD_low_list[0]]
    HD3_value = [HD_High_list[1], HD_low_list[1]]
    HD4_value = [HD_High_list[2], HD_low_list[2]]
    HD5_value = [HD_High_list[3], HD_low_list[3]] 
       
    #Set spur image locations: Fs-spur locations = spur image location
    Ny_HD2 = [Fs-HDlist[0][0], Fs-HDlist[0][1]]
    Ny_HD3 = [Fs-HDlist[1][0], Fs-HDlist[1][1]]
    Ny_HD4 = [Fs-HDlist[2][0], Fs-HDlist[2][1]]
    Ny_HD5 = [Fs-HDlist[3][0], Fs-HDlist[3][1]]
    
    #Create array to check if list values are greater than Nyquist
    #This is used to set spurs greater than Nyquist = Nyquist, for plotting purposes
    
    HD_NY_High = [Fs-HDlist[0][1],Fs-HDlist[1][1],Fs-HDlist[2][1],Fs-HDlist[3][1]]
    
    
    HD_High_NY_list=[]
    
    for i in HD_NY_High:
        if i > Fs :
            i = Fs
            temp_list = (i)
            #Update list with calculated values
            HD_High_NY_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
            #Update list with calculated values
            HD_High_NY_list.append(temp_list)
    
    #Create array to check if list values are less than 0
    #This is used to set spurs less than 0 = 0, for plotting purposes
    
    HD_NY_low = [Fs-HDlist[0][0],Fs-HDlist[1][0],Fs-HDlist[2][0],Fs-HDlist[3][0]]
    
   
    HD_low_NY_list=[]
    
    for i in HD_NY_low:
        if i < Nyquist:
            i = Nyquist
            temp_list = (i)
            #Update list with calculated values
            HD_low_NY_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
            #Update list with calculated values
            HD_low_NY_list.append(temp_list)

    
    #Create spur values with relative list values
    HD2_Ny_value = [HD_High_NY_list[0], HD_low_NY_list[0]]
    HD3_Ny_value = [HD_High_NY_list[1], HD_low_NY_list[1]]
    HD4_Ny_value = [HD_High_NY_list[2], HD_low_NY_list[2]]
    HD5_Ny_value = [HD_High_NY_list[3], HD_low_NY_list[3]] 
    
    #Check to see if spurs overlap the signal/signal image

    if (fin_low <= HD2_value[0])  and  (fin_high >= HD2_value[1]):
        print("HD2 overlaps")
    if (fin_low <= HD3_value[0])  and  (fin_high >= HD3_value[1]):
        print("HD3 overlaps")
    if (fin_low <= HD4_value[0])  and  (fin_high >= HD4_value[1]):
        print("HD4 overlaps")
    if (fin_low <= HD5_value[0])  and  (fin_high >= HD5_value[1]):
        print("HD5 overlaps")
        
    if (fin_low <= HD2_value[1])  and  (fin_high >= HD2_value[0]):
        print("HD2 overlaps")
    if (fin_low <= HD3_value[1])  and  (fin_high >= HD3_value[0]):
        print("HD3 overlaps")
    if (fin_low <= HD4_value[1])  and  (fin_high >= HD4_value[0]):
        print("HD4 overlaps")
    if (fin_low <= HD5_value[1])  and  (fin_high >= HD5_value[0]):
        print("HD5 overlaps")     
        
    #Return spurs and their images, return each individual value so data frame arrays can be created later
    return (HD_High_list[0], HD_low_list[0],HD_High_list[1], HD_low_list[1],
            HD_High_list[2], HD_low_list[2],HD_High_list[3], HD_low_list[3],
            HD_High_NY_list[0], HD_low_NY_list[0],HD_High_NY_list[1],
            HD_low_NY_list[1],HD_High_NY_list[2], HD_low_NY_list[2],
            HD_High_NY_list[3], HD_low_NY_list[3])

In [8]:
def Plot_RFDAC(Fs,Signal_center,Signal_Bandwidth):
    
    #Call functins to get input signal and signal image, spurs and spur images.
    Input_plot = DAC_Input_signal_calculator(Fs, Signal_center, Signal_Bandwidth)
    Spur_plot = DAC_Spur_Calculator(Fs,Signal_center,Signal_Bandwidth)
    Nyquist = Fs/2
    
    #y-coordinates
    n1 = [5]
    n2 = [2,2]
    n3 = [3,3]
    n4 = [4,4]
    n5 = [5,5]
    nn = [5,0]
    
    #Create list for data frame
    Spur_data = [Spur_plot[0],Spur_plot[1],Spur_plot[2],Spur_plot[3],Spur_plot[4],Spur_plot[5],Spur_plot[6],Spur_plot[7],
                 Spur_plot[8],Spur_plot[9],Spur_plot[10],Spur_plot[11],Spur_plot[12],Spur_plot[13],Spur_plot[14],Spur_plot[15]]
    
    #Data for plot
    input_zone_2 = [Input_plot[2],Input_plot[3]]
    input_zone_1 = [Input_plot[0],Input_plot[1]]
    HD2_value = [Spur_plot[0], Spur_plot[1]]
    HD3_value = [Spur_plot[2], Spur_plot[3]]
    HD4_value = [Spur_plot[4], Spur_plot[5]]
    HD5_value = [Spur_plot[6], Spur_plot[7]] 
    HD2_Ny_value = [Spur_plot[8], Spur_plot[9]]
    HD3_Ny_value = [Spur_plot[10], Spur_plot[11]]
    HD4_Ny_value = [Spur_plot[12], Spur_plot[13]]
    HD5_Ny_value = [Spur_plot[14], Spur_plot[15]] 
                 
    
    #x data
    spur_plot = [HD2_value,HD3_value,HD4_value,HD5_value,HD2_Ny_value,HD3_Ny_value,HD4_Ny_value,HD5_Ny_value]
     
    #y data
    y_data = [n2,n3,n4,n5,n2,n3,n4,n5]
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Harmonic number', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting
    spurs = Lines(x=spur_plot,y=y_data,scales={'x': x_sc, 'y': y_sc}, colors=['cyan','pink','purple','yellow','cyan','pink','purple','yellow'])
    Fin_horz_plot = Lines(x=input_zone_2,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert1_plot = Lines(x=[input_zone_2[0],input_zone_2[0]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert2_plot = Lines(x=[input_zone_2[1],input_zone_2[1]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_horz_plot_alias = Lines(x=input_zone_1,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert1_plot_alias = Lines(x=[input_zone_1[0],input_zone_1[0]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert2_plot_alias = Lines(x=[input_zone_1[1],input_zone_1[1]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Nyquist_plot = Lines(x=[Nyquist,Nyquist],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['red'])
    Fs_plot = Lines(x=[Fs,Fs],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['red'])
    
    
    #Create tooltips for hover
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency (MHz)', 'harmonic number'])
    tt_fim = Tooltip(fields=['x'], formats=[''], labels=['2nd Nyquist signal frequency(MHz)'])
    tt3 = Tooltip(fields=['x'], formats=[''], labels=['1st Nyquist Signal Frequency (MHz)'])
    Nyquisttt = Tooltip(fields=['x'], formats=[''], labels=['Nyquist Frequency (MHz)'])
    Fs_tt = Tooltip(fields=['x'], formats=[''], labels=['Sampling Frequency (MHz)'])
    
    #Create markers for data points
    spurs.marker='square'
    Fin_horz_plot.marker='square'
    Fin_horz_plot_alias.marker='square'
    Nyquist_plot.marker='square'
    Fs_plot.marker='square'
    #Plot
    fig = Figure(title='RFDAC frequency plan',axes=[ax_x, ax_y], marks=[Fin_horz_plot_alias,Fin_vert1_plot_alias,
                                                                        Fin_vert2_plot_alias,Fin_horz_plot,Fin_vert1_plot, 
                                                                        Fin_vert2_plot,spurs,Nyquist_plot,Fs_plot])
    
    #Match tooltips with relevant data parameter
    spurs.tooltip=tt
    Fin_horz_plot.tooltip=tt_fim
    Fin_horz_plot_alias.tooltip=tt3
    Nyquist_plot.tooltip=Nyquisttt
    Fs_plot.tooltip = Fs_tt
    tools=Toolbar(figure=fig)

    #Show figure
    display(fig,tools)
    
    #Create data frame for numerical values
    frame1={'':['Fin','HD2','HD3','HD4','HD5'],
                '1st Nyquist zone frequency (MHz)':[input_zone_1[0],Spur_data[1],Spur_data[3],Spur_data[5],Spur_data[7]],
                '1st Nyquist zone frequency (MHz)':[input_zone_1[1],Spur_data[0],Spur_data[2],Spur_data[4],Spur_data[6]],
                '2nd Nyquist zone frequency (MHz)':[input_zone_2[1],Spur_data[8],Spur_data[10],Spur_data[12],Spur_data[14]],
                '2nd Nyquist zone frequency (MHz)':[input_zone_2[0],Spur_data[9],Spur_data[11],Spur_data[13],Spur_data[15]]}
       
    
    tempFrame = pd.DataFrame(data=frame1)
    tempFrame.set_index('',inplace = True)    

    #Create button function
    def clicked(b):
    
    
        with out:
            clear_output(wait=True)
            display(tempFrame)
            
    out = widgets.Output()
    display(out)

    run_button = widgets.Button(
    description = 'Display numbers'
        )
    run_button.on_click(clicked)
    display(run_button)    


In [9]:
#Create sliders for selecting input information

Fs_slider = widgets.FloatSlider(value=4000,min=1000,max=4000,step=10,description='Fs:')
Signal_Center_slider= widgets.FloatSlider(value=3500,min=100,max=4000,step=50,description='Signal centre:')
Signal_Bandwidth = widgets.FloatSlider(value=100,min=50,max=400,step=50,description='Bandwidth:')

#Create text boxs for selecting input information
Fs_text = widgets.BoundedFloatText(value=4000,min=1000,max=4000,step=0.1,description='Fs:',continuous_update = False)
Signal_Center_text = widgets.BoundedFloatText(value=3500,min=100,max=4000,step=0.1,description='Signal centre:',continuous_update = False)
Signal_bandwidth_text = widgets.BoundedFloatText(value=50,min=50,max=400,step=0.1,description='Bandwidth:',continuous_update = False)

#Link values of text an sliders
l = widgets.link((Fs_slider, 'value'), (Fs_text, 'value'))
l = widgets.link((Signal_Center_slider, 'value'), (Signal_Center_text , 'value'))
l = widgets.link((Signal_Bandwidth, 'value'), (Signal_bandwidth_text, 'value'))

#Display layout of widgets
left_box = VBox([Fs_slider, Signal_Center_slider, Signal_Bandwidth])
right_box = VBox([Fs_text, Signal_Center_text, Signal_bandwidth_text])
sel=HBox([left_box, right_box])
display(sel)

#Interactive plot allows widgets to interact with plot function
out = widgets.interactive_output(Plot_RFDAC, {'Fs': Fs_slider, 'Signal_center': Signal_Center_slider, 'Signal_Bandwidth': Signal_Bandwidth})
display(out)

Output()

#  3) RF-ADC Frequency chart

The following chart provides the relevant information required for the RFADC process.
The chart displays the input signal and relevant spur information over the 1st Nyquist zone. 
Remember that due to aliasing all information can be displayed in Nyquist zone 1. 
Try to avoid overlap between the input signal and the harmonic spurs. HD2 and HD3 will degrade the signal the worst so it essential they do not overlap

In [10]:
#This function is used to calculate the frequency of the input signal in the 1st Nyquist zone
#Requires 3 input arguments, set from sliders
#Fs = Sampling Frequency, Signal_center = location of center of signal, Signal_bandwidth = Bandwidth of signal

def Input_signal_calculator(Fs, Signal_center, Signal_Bandwidth):
    
    #Calculate low and high bound of the signal
    Fin_low = Signal_center - (Signal_Bandwidth/2)
    Fin_High = Signal_center + (Signal_Bandwidth/2)
    
    #Calculate Nyquist rate
    Nyquist = float(Fs/2)
    
    #Set variable K depending on location of input signal 
    #K used in alias equation for folding signal back to 1st Nyquist zone
    if Fin_High <= Nyquist:
        K = 0
        
    elif Fin_High <= Fs + Nyquist:
        K = 1
    
    elif Fin_High <= 2*Fs + Nyquist:
        K = 2
        
    elif Fin_High <= 3*Fs + Nyquist:
        K = 3
    
    elif Fin_High <= 4*Fs + Nyquist:
        K = 4
    elif Fin_High <= 5*Fs + Nyquist:
        K = 5
        
    else:
        K = 6        
     
    
    #Calculate input alias
    Low_Input = (Fin_low - Fs*K)
    High_Input = (Fin_High - Fs*K)
    
    #Mirror input if value is negative
    if High_Input < 0:
        High_Input_return = Low_Input*-1
        Low_Input_return = High_Input*-1  
    
    #If signal lies in 2nd Nyquist zone then shift to first
    
    elif Low_Input > Nyquist:
        temp = High_Input - Nyquist
        Low_Input_return = Nyquist - temp
        temp2 = Low_Input - Nyquist
        High_Input_return = Nyquist - temp2
        
    #Return signal in first Nyquist zone
    else: 
        Low_Input_return = Low_Input
        High_Input_return = High_Input
 
    #If input signal reaches boundaries of 1st Nyquist zone, set equal to relative boundary
    if High_Input_return > Nyquist:
        High_Input_return = Nyquist
        
    if Low_Input_return < 0:
        Low_Input_return = 0
     
    #Failsafe against very low sampling frequency with high signal centre
    if High_Input_return < 0:
        High_Input_return = High_Input_return + Fs
        
    #Return the input values
    return [Low_Input_return, High_Input_return]

In [11]:
#This function is used to calculate the harmonic spurs relative to the input signal
#Requires 3 input arguments, set from sliders
#Fs = Sampling Frequency, Signal_center = location of center of signal, Signal_bandwidth = Bandwidth of signal

def Spur_Calculator(Fs, Signal_center, Signal_Bandwidth):

    #Call input signal function
    Input_info = Input_signal_calculator(Fs,Signal_center,Signal_Bandwidth)
    
    #Calculate Nyquist rate
    Nyquist = Fs/2
    
    #Set frequency bounds to returned values from function
    fin_low = Input_info[0]
    fin_high = Input_info[1]
    
    #Craete list for storing spur values
    HDlist=[]
    
    #Iterate through for loop to calculate spurs HD2 uptop HD5
    for n in range (2,6,1):
        
        #Calculate K vairable for folding spur signal back to first nyquist zone
        if fin_high*n <= Nyquist:
            K = 0
        
        elif fin_high*n <= Fs + Nyquist:
            K = 1
    
        elif fin_high*n <= 2*Fs + Nyquist:
            K = 2
        
        elif fin_high*n <= 3*Fs + Nyquist:
            K = 3
        
        elif fin_high*n <= 4*Fs + Nyquist:
            K = 4
            
        elif fin_high*n <= 5*Fs + Nyquist:
            K = 5
            
        elif Fin_High <= 6*Fs + Nyquist:
            K = 6
        else:
            K = 7

        
        #Calculate spur locations
        HDn_high = (fin_high *n) - (Fs*K)
        HDn_low = (fin_low *n) - (Fs*K)
        
        #IF HDn is negative, mirror result 
        if HDn_high <= 0:        

                    Mirror_HDn_low = HDn_low * -1;
                    Mirror_HDn_high = HDn_high * -1;
        
                    temp_list = (Mirror_HDn_low,Mirror_HDn_high) 
                    #Update list with calculated values 
                    HDlist.append(temp_list) 

        
        else:
                    temp_list = (HDn_high,HDn_low)
                    #Update list with calculated values
                    HDlist.append(temp_list)        
    
    #Create array to check if list values are greater than Nyquist
    #This is used to set spurs greater than Nyquist = Nyquist, for plotting purposes
    
    HD_High = [HDlist[0][0],HDlist[1][0],HDlist[2][0],HDlist[3][0]]
    
    HD_High_list=[]
    
    for i in HD_High:
        if i > Nyquist:
            i = Nyquist
            temp_list = (i)
            #Update list with calculated values
            HD_High_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
            #Update list with calculated values
            HD_High_list.append(temp_list)
    
    #Create array to check if list values are less than 0
    #This is used to set spurs less than 0 = 0, for plotting purposes
    
    HD_low = [HDlist[0][1],HDlist[1][1],HDlist[2][1],HDlist[3][1]]
    
    HD_low_list=[]
    
    for i in HD_low:
        if i < 0:
            i = 0
            temp_list = (i)
            #Update list with calculated values
            HD_low_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
            #Update list with calculated values
            HD_low_list.append(temp_list)

    #Create spur values with relative list values
    HD2_value = [HD_High_list[0], HD_low_list[0]]
    HD3_value = [HD_High_list[1], HD_low_list[1]]
    HD4_value = [HD_High_list[2], HD_low_list[2]]
    HD5_value = [HD_High_list[3], HD_low_list[3]] 
    
    
    #return spur values
    return [HD2_value[0],HD2_value[1],HD3_value[0],HD3_value[1],HD4_value[0],HD4_value[1],HD5_value[0],HD5_value[1]]



In [12]:
def overlap_calculator(Fs, Signal_center, Signal_Bandwidth):
    
    #Call input signal function
    Input_info = Input_signal_calculator(Fs,Signal_center,Signal_Bandwidth)
    
    #Calculate Nyquist rate
    Nyquist = Fs/2
    
    #Set frequency bounds to returned values from function
    fin_low = Input_info[0]
    fin_high = Input_info[1]

    spurs = Spur_Calculator(Fs,Signal_center,Signal_Bandwidth)
    
    #This section of code checks to see if the spur locations overlap the input signal
    #If true then the overlap will be flagged in the output
    
    if (fin_low <= spurs[0])  and  (fin_high >= spurs[1]):
        print("HD2 overlaps")
    if (fin_low <= spurs[2])  and  (fin_high >= spurs[3]):
        print("HD3 overlaps")
    if (fin_low <= spurs[4])  and  (fin_high >= spurs[5]):
        print("HD4 overlaps")
    if (fin_low <= spurs[6])  and  (fin_high >= spurs[7]):
        print("HD5 overlaps")
    
    return()

In [13]:
def calculate_interleaved_spur(Fs, Signal_center, Signal_Bandwidth):
    
    # Call spur calculator
    
    spurs = Spur_Calculator(Fs,Signal_center,Signal_Bandwidth)

    #Calculate Nyquist
    Nyquist = (Fs/2)
    
    #Sampling frequency/interleaved factors
    
    Fs2 = float(Fs/2)
    Fs4 = float(Fs/4)
    Fs8 = float(Fs/8)

    #Calculate all interleaved spurs
    #Lower bound
    #Higher bound
    Fs2m2_low = Fs2 - spurs[0]
    Fs2m2_high = Fs2 - spurs[1]

    Fs4m2_low = Fs4 - spurs[0]
    Fs4m2_high = Fs4 - spurs[1]

    Fs4p2_low = Fs4 + spurs[0]
    Fs4p2_high = Fs4 + spurs[1]

    Fs2m3_low = Fs2 - spurs[2]
    Fs2m3_high = Fs2 - spurs[3]

    Fs4m3_low = Fs4 - spurs[2]
    Fs4m3_high = Fs4 - spurs[3]

    Fs4p3_low = Fs4 + spurs[2]
    Fs4p3_high = Fs4 + spurs[3]

    Fs8m2_low = Fs8 - spurs[0]
    Fs8m2_high = Fs8 - spurs[1]
    
    Fs8p2_low = Fs8 + spurs[0]
    Fs8p2_high = Fs8 + spurs[1]

    Fs8m3_low = Fs8 - spurs[2]
    Fs8m3_high = Fs8 - spurs[3]
    
    Fs8p3_low = Fs8 + spurs[2]
    Fs8p3_high = Fs8 + spurs[3]
    
    #Store all interleaved values in array
    int_array = [Fs2m2_low,Fs2m2_high,Fs4m2_low,Fs4m2_high,Fs4p2_low,Fs4p2_high,Fs2m3_low,Fs2m3_high,Fs4m3_low,Fs4m3_high,
                 Fs4p3_low,Fs4p3_high,Fs8m2_low,Fs8m2_high,Fs8p2_low,Fs8p2_high,Fs8m3_low,Fs8m3_high,Fs8p3_low,Fs8p3_high]
   
    #Create array for storing
    new_int_array=[20]
    
    #Step through array in even steps
    for i in range(10):
        i = i*2
    
        # When over range break statement
        if i > 19:
            break
        
        #Use i and i+1 for boundaries. 
        #Both over Nyquist so calculate in first Nyquist zone
        if int_array[i] > Nyquist and int_array[i+1] > Nyquist:
            temp = int_array[i] - Nyquist
            temp_low = Nyquist - temp
            temp = int_array[i+1] - Nyquist
            temp_high = Nyquist - temp
            #Update list with calculated values
            new_int_array.insert(i,temp_low)
            new_int_array.insert(i+1,temp_high)
            
        #Both less than 0 so mirror into first Nyquist    
        elif int_array[i+1] < 0 and int_array[i] < 0:
            temp = int_array[i]*-1
            temp_high = int_array[i+1]*-1
            #Update list with calculated values
            new_int_array.insert(i,temp)
            new_int_array.insert(i+1,temp_high)
            
        #If lower bound is below Nyquist and higher bound is above set higher to Nyquist

        elif int_array[i] < Nyquist and int_array[i+1] > Nyquist:
            temp = int_array[i]
            temp_high = Nyquist 
            #Update list with calculated values
            new_int_array.insert(i,temp)
            new_int_array.insert(i+1,temp_high)
            
        elif int_array[i+1] < Nyquist and int_array[i] > Nyquist:
            temp = int_array[i+1]
            temp_high = Nyquist 
            #Update list with calculated values
            new_int_array.insert(i,temp_high)
            new_int_array.insert(i+1,temp)
  
        elif int_array[i+1] < 0 and int_array[i] > 0:
                temp = int_array[i]
                temp_high  = 0
                #Update list with calculated values
                new_int_array.insert(i,temp)
                new_int_array.insert(i+1,temp_high)
                
        elif int_array[i] < 0 and int_array[i+1] >= 0:
                temp = int_array[i+1]
                temp_low  = 0
                #Update list with calculated values
                new_int_array.insert(i,temp_low)
                new_int_array.insert(i+1,temp)
        else:
            
            temp_list = int_array[i]
            temp_list2 = int_array[i+1]
            #Update list with calculated values
            new_int_array.insert(i,temp_list)
            new_int_array.insert(i+1,temp_list2)
          
    return(new_int_array)


In [14]:
#This function plots the RFADC chart
#Requires 3 input arguments, set from sliders
#Fs = Sampling Frequency, Signal_center = location of center of signal, Signal_bandwidth = Bandwidth of signal

def Plot_RFADC(Fs,Signal_center,Signal_Bandwidth):
    
    #Call function to calculate input signal
    Input_plot = Input_signal_calculator(Fs, Signal_center, Signal_Bandwidth)
    
    #Call function to calculate the harmonic spurs
    Spur_plot = Spur_Calculator(Fs,Signal_center,Signal_Bandwidth)
    
    Interleaved_plot = calculate_interleaved_spur(Fs, Signal_center, Signal_Bandwidth)
  
    overlap = overlap_calculator(Fs, Signal_center, Signal_Bandwidth)
    #Calculate Nyquist rate
    Nyquist = Fs/2
    
    #Set y-coordinates for plot
    n1 = [5]
    n2 = [2,2]
    n3 = [3,3]
    n4 = [4,4]
    n5 = [5,5]
    nn = [5,0]
    
    #Set y-coordinates for interleaved spurs
    ni2 = [2.5,2.5]
    ni3 = [3.5,3.5]

    #Create arrays for plotting
    
    #Spur locations for data frame
    spur_data = [Spur_plot[0],Spur_plot[1],Spur_plot[2],Spur_plot[3],Spur_plot[4],Spur_plot[5],Spur_plot[6],Spur_plot[7]]
    
    int_spur_data = [Interleaved_plot[0],Interleaved_plot[1],Interleaved_plot[2],Interleaved_plot[3],
                     Interleaved_plot[4],Interleaved_plot[5],Interleaved_plot[6],Interleaved_plot[7],
                     Interleaved_plot[8],Interleaved_plot[9],Interleaved_plot[10],Interleaved_plot[11],
                     Interleaved_plot[12],Interleaved_plot[13],Interleaved_plot[14],Interleaved_plot[15],
                     Interleaved_plot[16],Interleaved_plot[17],Interleaved_plot[18],Interleaved_plot[19]]
   
    #Input signal 
    input_data = [Input_plot[0],Input_plot[1]]
    
    #y axis, used to place spurs at relative height
    y_data = [n2, n3, n4, n5]
    
    #Create for plotting spurs
    HD2_value = [Spur_plot[0], Spur_plot[1]]
    HD3_value = [Spur_plot[2], Spur_plot[3]]
    HD4_value = [Spur_plot[4], Spur_plot[5]]
    HD5_value = [Spur_plot[6], Spur_plot[7]] 
    spur_plot2 = [HD2_value,HD3_value,HD4_value,HD5_value]
    
    # Plot interleaves spurs [Bound 1 , Bound 2]
    
    fsd2m2= [Interleaved_plot[0],Interleaved_plot[1]]     
    fsd4m2= [Interleaved_plot[2],Interleaved_plot[3]]   
    fsd4p2= [Interleaved_plot[4],Interleaved_plot[5]] 
    fsd2m3= [Interleaved_plot[6],Interleaved_plot[7]] 
    fsd4m3= [Interleaved_plot[8],Interleaved_plot[9]]       
    fsd4p3= [Interleaved_plot[10],Interleaved_plot[11]]    
    fsd8p2= [Interleaved_plot[12],Interleaved_plot[13]]    
    fsd8m2= [Interleaved_plot[14],Interleaved_plot[15]]
    fsd8p3= [Interleaved_plot[16],Interleaved_plot[17]]   
    fsd8m3= [Interleaved_plot[18],Interleaved_plot[19]]
    
    #Ploting locations for interleaved spurs
    
    int_spur_plot = [fsd2m2,fsd4m2,fsd4p2,fsd2m3,fsd4m3,fsd4p3,fsd8p2,fsd8m2,fsd8p3,fsd8m3]
    int_y_plot = [ni2,ni2,ni2,ni3,ni3,ni3,ni2,ni2,ni3,ni3]
    
    #Set scale to linear
    x_sc = LinearScale()
    y_sc = LinearScale()
    
    #Label axis
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Harmonic number', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting data
    spurs = Lines(x=spur_plot2,y=y_data,scales={'x': x_sc, 'y': y_sc}, colors=['cyan','pink','purple','yellow'],enable_move=False)
    int_spurs = Lines(x=int_spur_plot,y=int_y_plot,scales={'x': x_sc, 'y': y_sc}, colors=['orange','orange','orange','green','green',
                                                                                          'green','orange','orange','green','green'],
                      line_style='dashed',enable_move=False)  
    Fin_horz_plot = Lines(x=input_data,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert1_plot = Lines(x=[input_data[0],input_data[0]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert2_plot = Lines(x=[input_data[1],input_data[1]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_horz_plot_alias = Lines(x=input_data,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Nyquist_plot = Lines(x=[Nyquist,Nyquist],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['red'])
    
    #Create tooltips for displaying data on hover
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency (MHz)', 'harmonic number'])
    tt_fim = Tooltip(fields=['x'], formats=[''], labels=['Signal Frequency image (MHz)'])
    Nyquisttt = Tooltip(fields=['x'], formats=[''], labels=['Nyquist Frequency (MHz)'])
    tt_int_spurs = Tooltip(fields=['x'], formats=[''], labels=['Frequency (MHz)'])
    
    #Create data tip shapes
    spurs.marker='square'
    Fin_horz_plot.marker='square'
    int_spurs.marker='square'
    Nyquist_plot.marker='square'
    int_spurs.marker='square'
    
    #Plot figure
    fig = Figure(title='RFADC frequency plan',axes=[ax_x, ax_y], marks=[Fin_horz_plot,Fin_vert1_plot, Fin_vert2_plot,spurs,
                                                                        Nyquist_plot, int_spurs])
    spurs.tooltip=tt
    int_spurs.tooltip = tt_int_spurs
    Fin_horz_plot.tooltip=tt_fim
    Nyquist_plot.tooltip=Nyquisttt
    tools=Toolbar(figure=fig)
    
    
    #Display figure and relevant tools
    display(fig,tools)
    
    #Create data frame    
    frame1={'':['Fin','HD2','HD3','HD4','HD5'],
                'Frequency high (MHz)':[input_data[1],spur_data[0],spur_data[2],spur_data[4],spur_data[6]],
                'Frequency low (MHz)':[input_data[0],spur_data[1],spur_data[3],spur_data[5],spur_data[7]]}
       
    
    tempFrame = pd.DataFrame(data=frame1)
    tempFrame.set_index('',inplace = True)    
    tempFrame.to_csv('ADC.csv', sep=',',index=False)

    #Create function for button action (Display numbers button)
    def clicked(b):
    
        with out:
            clear_output(wait=True)
            display(tempFrame)
    
    #Show button
    out = widgets.Output()
    display(out)

    run_button = widgets.Button(
        description = 'Display numbers'
        )
    run_button.on_click(clicked)
    display(run_button)    


In [15]:
#Create sliders for selecting input information

Fs_slider = widgets.FloatSlider(value=4000,min=1000,max=4000,step=50,description='Fs:')
Signal_Center_slider= widgets.FloatSlider(value=3500,min=100,max=4000,step=50,description='Signal Centre:')
Signal_Bandwidth = widgets.FloatSlider(value=100,min=50,max=400,step=50,description='Bandwidth:')

#Create text boxs for more accurate entries 
Fs_text = widgets.BoundedFloatText(value=4000,min=1000,max=4000,step=0.1,description='Fs:',continuous_update = False)
Signal_Center_text = widgets.BoundedFloatText(value=3500,min=100,max=4000,step=0.1,description='Signal centre:',continuous_update = False)
Signal_bandwidth_text = widgets.BoundedFloatText(value=100,min=50,max=400,step=0.1,description='Bandwidth:',continuous_update = False)

#link sliders and text boxs so user can use whichever they please
l = widgets.link((Fs_slider, 'value'), (Fs_text, 'value'))
l = widgets.link((Signal_Center_slider, 'value'), (Signal_Center_text , 'value'))
l = widgets.link((Signal_Bandwidth, 'value'), (Signal_bandwidth_text, 'value'))

#Display layout
left_box = VBox([Fs_slider, Signal_Center_slider, Signal_Bandwidth])
right_box = VBox([Fs_text, Signal_Center_text, Signal_bandwidth_text])
sel=HBox([left_box, right_box])
display(sel)

#Interactive output creates link between plotting function and widget inputs
out = widgets.interactive_output(Plot_RFADC, {'Fs': Fs_slider, 'Signal_center': Signal_Center_slider, 'Signal_Bandwidth': Signal_Bandwidth})
display(out)

Output()

# 4) DDC Spectrum

The following spectrum displays the relevant information for the Digital down conversion process. Note the values displayed are the decimated values. Any values exceeding the decimated Nyquist value are folded back until they are present in the display band.

In [16]:
#Create function to calculate decimated values for frequency in and frequency image

def Equation_calculator(Fs, Frequency_in, NCO, Decimation , Interleaved_ADC):
    
    Nyquist = Fs/2 #Calculate Nyquist value
    
    # Calculate K variable, K increases as conditions are met
    
    if Frequency_in < Nyquist:
        K=0
        
    elif Frequency_in > Nyquist:
        K = 1
        
    elif Frequency_in > Fs + Nyquist:
        K = 2
    
    elif Frequency_in > 2*Fs + Nyquist:
        K = 3
        
    elif Frequency_in > 3*Fs + Nyquist:
        K = 4
        
    else:
        K = 5
        
    ## Potentially add error statement if input frequency reaches unrealistic value 
    
    #Calculate input alias before NCO shift
    Input = (Frequency_in - Fs*K)
    
    #Mirror input if value is negative
    if Input < 0:
        Input=Input*-1
    
    #Calculate image, needs to be after mirrored input value is calculated 
    Input_Image = Fs - Input
    
    if Input_Image < 0:
        Input_Image  = Input_Image *-1
            
    #Shift input by NCO
    Input_NCO = (Input) - NCO
    Input_Image_NCO = (Input_Image) - NCO
    
    #Calculate final Answer for input alias 
    Answer = Input_NCO + (Interleaved_ADC * Fs/Decimation)
    Answer2 = Input_Image_NCO + (Interleaved_ADC * Fs/Decimation)
    
    if Answer < 0:
        Answer = Answer * -1
    
    if Answer2 < 0:
        Answer2 = Answer2 * -1
    
    #Use decimated Nyquist and Fs values for calculating signal in 1st decimated Nyquist zone
    Decimated_Nyquist = Nyquist/Decimation
    Decimated_Fs = Fs/Decimation

    #Use following process to continuously subtract decimated FS until value is displayed in first decimated Nyquist zone

    stopprocess = False

    while stopprocess == False:
            if Answer > Decimated_Nyquist:
                Answer = Answer-Decimated_Fs
            
            #Set to true when value is below decimated Nyquist
                if Answer <= Decimated_Nyquist and Answer > -1*Decimated_Nyquist:
                    stoprocess = True
                    Answer = Answer
                    break
                
    stopprocess2 = False
        
    while stopprocess2 == False:
            if Answer2 > Decimated_Nyquist:
                Answer2 = Answer2-Decimated_Fs
            
            #Set to true when value is below decimated Nyquist
                if Answer2 <= Decimated_Nyquist and Answer2 > -1*Decimated_Nyquist:
                    stoprocess2 = True
                    Answer2=Answer2
                    break
                    
    return [Answer,Answer2]

In [17]:
#Create functino for calculating spurs for DDC

def DDC_spur_calculator(Fs, Frequency_in, NCO, Decimation , Interleaved_ADC):
    
    
    Nyquist = Fs/2 #Calculate Nyquist value
    HDlist = [] #Create a list for storing loop values
    
    # Calculate K variable, K increases as conditions are met
    
    for n in range (2,4,1):
       
        
        if Frequency_in*n < Nyquist:
            K = 0
        
        elif Frequency_in*n <= Fs + Nyquist:
            K = 1
    
        elif Frequency_in*n <= 2*Fs + Nyquist:
            K = 2
        
        elif Frequency_in*n <= 3*Fs + Nyquist:
            K = 3
        
        else:
            K = 4

        hdn_orig = (Frequency_in*n) - (Fs*K)
        
        
        if hdn_orig < 0:
            hdn_orig = hdn_orig*-1
            
        hdn_image = Fs - hdn_orig
            
        if hdn_image < 0:
            hdn_image = hdn_image*-1
        
        #Calculate NCO shift
        hd_NCO = hdn_orig - NCO
        hd_image_nco = hdn_image - NCO
        
        #Calculate final Answer for input alias 
        Answer = hd_NCO + (Interleaved_ADC * Fs/Decimation)
        Answer2 = hd_image_nco + (Interleaved_ADC * Fs/Decimation)
        
        #Calculate Decimated values
        Decimated_Nyquist = Nyquist/Decimation
        Decimated_Fs = Fs/Decimation
    
        
        #Use following process to continuously subtract decimated FS until value is displayed in first decimated Nyquist zone
        stopprocess = False

        while stopprocess == False:
            if Answer > Decimated_Nyquist:
                Answer = Answer-Decimated_Fs
            
            #Set to true when value is below decimated Nyquist
                if Answer <= Decimated_Nyquist:
                    stoprocess = True
                    temp_list = (Answer)
                    HDlist.append(temp_list) 
                    Answer = 0
                    break
                

        stopprocess2 = False
        
        while stopprocess2 == False:
            if Answer2 > Decimated_Nyquist:
                Answer2 = Answer2-Decimated_Fs
            
            #Set to true when value is below decimated Nyquist
                if Answer2 <= Decimated_Nyquist:
                    stoprocess2 = True
                    temp_list = (Answer2)
                    HDlist.append(temp_list) 
                    Answer2 = 0
                    break
                
   
    #Values to return
    HD2_value = HDlist[0]
    HD3_value = HDlist[2]
    HD2_image_value = HDlist[1]
    HD3_image_value = HDlist[3]
    
    return [HD2_value, HD3_value, HD2_image_value, HD3_image_value]


In [18]:
#Plot DDC graph

def Plot_DDC(Fs, Frequency_in,NCO,HD2_noise,HD3_noise, Decimation, Interleaved_ADC):
    
    #Calcaulte nyquist and decimated nyquist
    Nyquist = Fs/2
    Decimated_Nyquist = Nyquist/Decimation
    
    #Call input calculator for frequency
    Frequency = Equation_calculator(Fs,Frequency_in,NCO,Decimation,Interleaved_ADC)
    
    #Call spur function
    HD = DDC_spur_calculator(Fs,Frequency_in,NCO, Decimation, Interleaved_ADC)

    #y value for spur noise selection
    y_plot_HD2 = [-1*HD2_noise,-100]
    y_plot_HD3 = [-1*HD3_noise,-100]

    HD2 = HD[0]
    HD3 = HD[1]
    HD2_image = HD[2]
    HD3_image = HD[3]
    
    #Create x data
    x_plot_HD2 = [HD2, HD2]
    x_plot_HD2_image = [HD2_image, HD2_image]
    x_plot_HD3 = [HD3, HD3]
    x_plot_HD3_image = [HD3_image, HD3_image]
    x_data_HD2 = [x_plot_HD2]
    x_data_HD3 = [x_plot_HD3]
    x_data_HD2_image = [x_plot_HD2_image]
    x_data_HD3_image = [x_plot_HD3_image]
    
    #Create y data
    y_data_HD2 = [y_plot_HD2]
    y_data_HD3 = [y_plot_HD3]
  
    f_in = [Frequency[0],Frequency[0]]
    f_image = [Frequency[1],Frequency[1]]
    
    fund_y = [0,-100]
    Decimated_Nyquist_plot = [Decimated_Nyquist,Decimated_Nyquist]
    Negative_Nyquist = [-1*Decimated_Nyquist,-1*Decimated_Nyquist]
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Amplitude(dB)', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting
    spur_HD2 = Lines(x=x_data_HD2,y=y_data_HD2,scales={'x': x_sc, 'y': y_sc}, colors=['cyan'])
    spur_HD3 = Lines(x=x_data_HD3,y=y_data_HD3,scales={'x': x_sc, 'y': y_sc}, colors=['pink'])
    spur_HD2_image = Lines(x=x_data_HD2_image,y=y_data_HD2,scales={'x': x_sc, 'y': y_sc}, colors=['cyan'])
    spur_HD3_image = Lines(x=x_data_HD3_image,y=y_data_HD3,scales={'x': x_sc, 'y': y_sc}, colors=['pink'])
    f_in = Lines(x=f_in,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    f_image= Lines(x=f_image,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    
    Decimated_Nyquist_plot = Lines(x=Decimated_Nyquist_plot,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'])
    Negative_Nyquist_plot = Lines(x=Negative_Nyquist,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'])
  
    #Tooltips
   
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency in (MHz)', 'dB level (dB)'])
    tt_image = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency in image (MHz)', 'dB level (dB)'])
    tt_HD2 = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['HD2 (MHz)', 'dB level (dB)'])
    tt_HD3 = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['HD3 (MHz)', 'dB level (dB)'])
    tt_HD2_image = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['HD2 image (MHz)', 'dB level (dB)'])
    tt_HD3_image = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['HD3 image (MHz)', 'dB level (dB)'])
    tt3 = Tooltip(fields=['x'], formats=[''], labels=['Decimated Nyquist Frequency (MHz)'])
    
    #Markers for data points
    spur_HD2.marker='square'
    spur_HD3.marker='square'
    spur_HD2_image.marker='square'
    spur_HD3_image.marker='square'
    f_in.marker='square'
    f_image.marker='square'
    Decimated_Nyquist_plot.marker = 'square'
    Negative_Nyquist_plot.marker = 'square'

    
    fig = Figure(title='DDC Spectrum',axes=[ax_x, ax_y], marks=[spur_HD2,spur_HD3,spur_HD2_image,spur_HD3_image,f_in,f_image,Decimated_Nyquist_plot,Negative_Nyquist_plot])
    
    #Link tooltips with relative data
    spur_HD2_image.tooltip=tt_HD2_image
    spur_HD3_image.tooltip=tt_HD3_image
    spur_HD2.tooltip=tt_HD2
    spur_HD3.tooltip=tt_HD3
    f_in.tooltip=tt
    f_image.tooltip=tt_image
    Decimated_Nyquist_plot.tooltip=tt3
    Negative_Nyquist_plot.tooltip=tt3
    tools=Toolbar(figure=fig)
        
    display(fig, tools)
    
    #Create data frame
    frame2={'':['Fin','Fin image','NCO','HD2','HD2 image','HD3','HD3 image'],
            'Frequency (MHz)':[Frequency[0],Frequency[1],NCO,HD[0],HD[2],HD[1],HD[3]]}
       
    tempFrame = pd.DataFrame(data=frame2)
    tempFrame.set_index('',inplace = True)    

    #Button for displaying data frame
    def clicked(b):
    
        with out:
            clear_output(wait=True)
            display(tempFrame)
            
    out = widgets.Output()
    display(out)

    run_button = widgets.Button(
        description = 'Display numbers'
        )
    run_button.on_click(clicked)
    display(run_button)    

In [19]:
Fs_slider = widgets.FloatSlider(value=4000,min=1000,max=4000,step=100,description='Fs:')
Signal_Center_slider = widgets.FloatSlider(value=3500,min=100,max=4000,step=50,description='Signal centre:')
NCO_slider = widgets.FloatSlider(value=0,min=-2000,max=2000, step = 50,description='NCO:')
HD2_slider = widgets.FloatSlider(value=10, min=0, max=100, step = 10,description='HD2 noise:')
HD3_slider = widgets.FloatSlider(value=10, min=0, max=100, step = 10,description='HD3 noise:')
#widgets.interact(Plot_DUC,Fs=Fs_slider,Frequency_in=Signal_Center_slider,NCO=NCO_slider,HD2_noise=HD2_slider,HD3_noise=HD3_slider)

#Create button widgets to control variables
Decimation_buttons = widgets.RadioButtons(
    options=[1,2,4,8],
    value=8,
    description='Decimation:',
    disabled=False
)

Interleaved_ADC_buttons = widgets.RadioButtons(
    options=[4,8],
    value=8,
    description='Int ADCs:',
    disabled=False
)


Fs_text = widgets.BoundedFloatText(value=4000,min=1000,max=4000,step=0.1,description='Fs:',continuous_update = False)
Signal_Center_text = widgets.BoundedFloatText(value=3500,min=100,max=4000,step=0.1,description='Signal centre:',continuous_update = False)
NCO_text = widgets.BoundedFloatText(value=0,min=-2000,max=2000,step=0.1,description='NCO:',continuous_update = False)
HD2_text = widgets.BoundedFloatText(value=10,min=0,max=100,step=0.1,description='HD2 noise:',continuous_update = False)
HD3_text = widgets.BoundedFloatText(value=10,min=0,max=100,step=0.1,description='HD3 noise:',continuous_update = False)
l = widgets.link((Fs_slider, 'value'), (Fs_text, 'value'))
l = widgets.link((Signal_Center_slider, 'value'), (Signal_Center_text , 'value'))
l = widgets.link((NCO_slider, 'value'), (NCO_text, 'value'))
l = widgets.link((HD2_slider, 'value'), (HD2_text, 'value'))
l = widgets.link((HD2_slider, 'value'), (HD2_text, 'value'))

left_box = VBox([Fs_slider, Signal_Center_slider, NCO_slider,HD2_slider,HD3_slider])
right_box = VBox([Fs_text, Signal_Center_text, NCO_text,HD2_text,HD3_text])
button_box = VBox([Interleaved_ADC_buttons,Decimation_buttons])

sel=HBox([left_box, right_box,button_box])

display(sel)

Fs= Fs_slider
Signal_center = Signal_Center_slider
NCO = NCO_slider


out = widgets.interactive_output(Plot_DDC, {'Fs': Fs, 'Frequency_in': Signal_center, 'NCO': NCO,
                                              'HD2_noise':HD2_slider,'HD3_noise':HD3_slider,'Decimation':Decimation_buttons,
                                            'Interleaved_ADC':Interleaved_ADC_buttons})
display(out)



Output()